# Задание 4

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [183]:
from collections import OrderedDict

class LruCache(object):
    def __init__(self, max_count):
        self.storage = OrderedDict()
        self.max_count = max_count

    def __getitem__(self, key):
        return self.storage.get(key)

    def __setitem__(self, key, value):
        if len(self.storage) >= self.max_count:
            self.storage.popitem(last=False)
        self.storage[key] = value

#### Реализуйте декоратор

In [184]:
import functools
from datetime import datetime

def cached(max_count):
    lc = LruCache(max_count)
    def decorator(func):
        @functools.wraps(func)
        def wrapper(n):
            if lc[n] != None:
                return lc[n] 
            else:
                result = func(n)
                lc[n] = result
                return result
        return wrapper
    return decorator

#### Проверьте использование декоратора

In [185]:
@cached(2)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

In [186]:
print(fact(10))

3628800


Как видим, результаты работы функции кэшируются

In [187]:
start = datetime.now()
fact(1000)
print(datetime.now() - start)

0:00:00.006180


In [188]:
start = datetime.now()
fact(1001)
print(datetime.now() - start)

0:00:00.000389


Содержание кэша можно посмотреть так:

In [189]:
#print(fact.__closure__[1].cell_contents.storage)
#print(fact.__closure__[1].cell_contents.storage.values())
print(fact.__closure__[1].cell_contents.storage.keys())

odict_keys([1000, 1001])


#### Сравните свою реализацию с lru_cache из functools

In [190]:
from functools import lru_cache

@lru_cache(2)
def fact2(n):
    if n < 2:
        return 1
    return fact(n-1) * n

In [191]:
start = datetime.now()
fact2(1000)
print(datetime.now() - start)

0:00:00.005691


In [192]:
start = datetime.now()
fact2(1001)
print(datetime.now() - start)

0:00:00.000198


Cache из functools работает примерно так же

### Дополнительное задание (0.2 балла)

Дополните декоратор @cached так, чтобы не пересчитывать функцию при изменения ее состояния (например, она использовала глобальную переменную)

#### В моей реализации декоратора пересчитывание функции не произойдет при изменении ее состояния

## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [193]:
def checked(*types):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for t in args:
                #print(t)
                if type(t) not in types:
                    raise TypeError("function '" + func.__name__ + "' got wrong typed arguments" )
            for key in kwargs.keys():
                #print(kwargs[key])
                #print(type(kwargs[key]))
                if type(kwargs[key]) not in types:
                    raise TypeError("function '" + func.__name__ + "' got wrong typed arguments" )
        return wrapper
    return decorator

#### Проверьте использование декоратора

In [194]:
from typing import List

@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    pass

In [195]:
#strange_func(6, [3], 2)
#strange_func(6, 3, 'a')
strange_func(4, a = 'd', c = [])

## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

In [196]:
import functools
from datetime import datetime
from __future__ import print_function
import sys

def logString(index, funcName, args, kwargs, result):
    return(str(index) + '   ' + 
           str(datetime.now().strftime('%Y-%m-%d')) + '   ' + 
           str(datetime.now().strftime('%H:%M:%S')) + '   ' + 
           str(funcName) + '   ' + 
           str(args) + '   '+ str(kwargs) + '  ' + str(result) + '\n')

class Log(object):
    index = 0
    def __init__(self, func):
        self.func = func

    def __call__(self, outStream, *args, **kwargs):
        result = self.func(*args, **kwargs)
        Log.index += 1
        if outStream == 1:
            print(logString(Log.index, self.func.__name__, args, kwargs, result), file=sys.stderr)
        elif type(outStream) == type('routeToFile'):
            with open(outStream, 'a+') as file:
                file.write(logString(Log.index, self.func.__name__, args, kwargs, result))
        else:
            print(logString(Log.index, self.func.__name__, args, kwargs, result), file=sys.stdout)
        return result

In [197]:
def Logger(stdout = 0):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            log = Log(func)
            log(stdout, *args, **kwargs)
        return wrapper
    return decorator

Logger("anyFileName.txt") пишет логи в файл "anyFileName.txt"

Logger(1) пишет логи в stderr

При остальных аргументах запись логов ведется в stdout

In [198]:
@Logger()
def f(n):
    sum = 0
    for i in range(n):
        sum += i
    return sum

@Logger(1)
def f1(n):
    sum = 0
    for i in range(n):
        sum += i
    return sum

@Logger("myLogger.txt")
def f2(n):
    sum = 0
    for i in range(n):
        sum += i
    return sum


In [199]:
f(7)

1   2018-12-12   23:44:30   f   (7,)   {}  21



In [200]:
f1(7)

2   2018-12-12   23:44:32   f1   (7,)   {}  21



In [201]:
f2(7)